In [55]:
%pylab inline
import pandas as pd
import sklearn as skl
import scipy
import math  
import numpy as np
from collections import defaultdict
import torch
from transformers import BertModel, BertTokenizerFast
from sklearn.metrics.pairwise import cosine_similarity

model_name = 'ai-forever/ruBert-base'
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
document = pd.read_excel("../News_SGU_31077_Processed_1.xlsx")
texts = document["News_Tokens"]

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\vadim\scoop\apps\python\3.12.6\Lib\site-packages\IPython\core\magics\pylab.py:166: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [56]:
print(len(texts))

31077


In [57]:
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import torch

word_embedding = defaultdict(list)
# texts = texts[:1000]

max_length = 512
stride = 256

for text in tqdm(texts, desc="get embeddings"):
    # Токенизация с word_ids для отслеживания исходных слов
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=max_length,
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=True,
    )

    chunk_inputs = {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "token_type_ids": inputs.get("token_type_ids"),
    }

    # Обрабатываем каждый чанк
    for i in range(inputs["input_ids"].shape[0]):
        current_input = {
            k: v[i].unsqueeze(0) for k, v in chunk_inputs.items() if v is not None
        }

        with torch.no_grad():
            outputs = model(**current_input)

        embeddings = outputs.last_hidden_state[0].numpy()
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][i])
        word_ids = inputs.word_ids(batch_index=i)  # Получаем индексы слов для каждого токена

        # Группируем эмбеддинги по словам
        current_id = None
        current_word = ""
        word_embeddings_temp = []
        for j, (token, word_id) in enumerate(zip(tokens, word_ids)):
            if word_id is not None: # Не служебный символ
                if word_id != current_id:
                    # Если началось новое слово, сохраняем предыдущее (если было)
                    if current_id is not None:
                        word_embedding[current_word].append(np.mean(word_embeddings_temp, axis=0))
                    current_id = word_id
                    current_word = token
                    word_embeddings_temp = [embeddings[j]]
                else:
                    # Добавляем эмбеддинг подтокена к текущему слову
                    current_word += token[2:]
                    word_embeddings_temp.append(embeddings[j])

        # Добавляем последнее слово в чанке
        if current_id is not None:
            word_embedding[current_word].append(np.mean(word_embeddings_temp, axis=0))

# Усредняем эмбеддинги для каждого слова (если слово встречается в нескольких чанках)
for word in word_embedding:
    word_embedding[word] = np.mean(word_embedding[word], axis=0)
# Теперь word_embedding содержит эмбеддинги целых слов (не подтокенов)
print(len(word_embedding))

get embeddings: 100%|██████████| 31077/31077 [3:34:58<00:00,  2.41it/s]  


71796


In [58]:
print(embeddings)

[[ 0.48035258  0.75337803  0.37968978 ...  0.53577435  0.89375055
   0.09614167]
 [ 0.9589678   0.11851829 -0.09660891 ...  0.20498708 -0.13616273
   0.46041542]
 [ 0.17332472 -0.24964423  0.1084326  ... -0.19036618 -0.6807928
   0.46571916]
 ...
 [-0.29190627  0.09819216  0.24448213 ... -0.06546737  0.06231318
   0.55827415]
 [ 0.03648516 -0.04748931  0.8381751  ... -0.16265407  0.08596082
  -0.05247702]
 [ 0.35626978  0.40936068 -0.37038222 ...  0.23060995  0.28282765
   0.16283448]]


In [59]:
words = list(word_embedding.keys())

In [60]:
embeddings_matrix = np.array([word_embedding[word] for word in words])

In [61]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(embeddings_matrix)

In [ ]:
def get_top_similar_words(word, top_n=10, threshold = 0.6):
    if word not in word_embedding:
        return []
    idx = words.index(word)
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_words = [(words[i], score) for i, score in sim_scores[1:top_n+1] if score > threshold]  # [1:] чтобы исключить само слово
    return top_words

print(get_top_similar_words("сгу"))
# Пример вывода: [("large", 0.92), ("huge", 0.89), ("enormous", 0.85), ...]

[('биф', 0.84560966), ('сгуоб', 0.8114157)]


In [63]:
result = []
for word in tqdm(words, desc="get_sim_word"):
  result.append([word, get_top_similar_words(word)])
df_result = pd.DataFrame(result, columns=["Word", "Most_Similar_Word"])
df_result.to_csv("sim-words5-bert.csv", index=False)

get_sim_word: 100%|██████████| 71796/71796 [3:01:55<00:00,  6.58it/s]  
